## Loading the data and libraries


In [16]:
import pandas as pd #requires library
#loaded raw and incremental datasets
raw_df = pd.read_csv("data/raw_data.csv")
inc_df = pd.read_csv("data/incremental_data.csv")

## T1:HANDLING MISSING VALUES

### FULL

In [17]:
# Before
print("Full - Missing Before:")
print(raw_df[['quantity', 'unit_price']].isnull().sum())

Full - Missing Before:
quantity      26
unit_price    35
dtype: int64


In [18]:
#fill missing quantity and price
raw_df['quantity'] = raw_df['quantity'].fillna(raw_df['quantity'].median())#decided to use median for filling missing values
raw_df['unit_price'] = raw_df['unit_price'].fillna(raw_df['unit_price'].median())#decided to use median for filling missing values

#after
print("Full - Missing After:")#this will show the number of missing values after filling
print(raw_df[['quantity', 'unit_price']].isnull().sum())#will show the number of missing values after filling

Full - Missing After:
quantity      0
unit_price    0
dtype: int64


no missing values

In [19]:
#fill missing customer name in full dataset
raw_df['customer_name'] = raw_df['customer_name'].fillna("Unknown")#decided to fill missing customer names with "Unknown" a placeholder value since it has only one missing value
raw_df['region'] = raw_df['region'].fillna("Unknown")#also filling missing region with "Unknown" since it has 25 missing values and it will not affect the analysis much


### INCREMENTAL

In [20]:
print("Incremental - Missing Before:")
print(inc_df[['customer_name', 'quantity']].isnull().sum())



Incremental - Missing Before:
customer_name    6
quantity         4
dtype: int64


In [21]:
# Fill with placeholder
inc_df['customer_name'] = inc_df['customer_name'].fillna("Unknown")#decided to fill missing customer names with "Unknown" a placeholder value since it won't affect the analysis much
inc_df['quantity'] = inc_df['quantity'].fillna(inc_df['quantity'].median())#filled missing quantity with median value

print("Incremental - Missing After:")
print(inc_df[['customer_name', 'quantity']].isnull().sum())# this will show the number of missing values after filling

Incremental - Missing After:
customer_name    0
quantity         0
dtype: int64


no missing values

In [22]:
#adding a placeholder for region
# Fill missing region in incremental with 'Unknown'
inc_df['region'] = inc_df['region'].fillna("Unknown")


## T2:CONVERTING ORDERDATE TO DATETIME

In [23]:
raw_df['order_date'] = pd.to_datetime(raw_df['order_date'], errors='coerce', dayfirst=True)#converting order_date to datetime format in the raw dataset and used dayfirst=True to ensure correct parsing of dates,the format i want
inc_df['order_date'] = pd.to_datetime(inc_df['order_date'], errors='coerce', dayfirst=True)#converting order_date to datetime format in the incremental dataset

print(raw_df.dtypes['order_date'])  #should return datetime64[ns]
print(inc_df.dtypes['order_date'])  #should return datetime64[ns]


datetime64[ns]
datetime64[ns]


## T3: CONVERTING UNITPRICE TO FLLOAT
earlier the unit price was an integer and since price usually contains decimals, hence leaving them as  integers could limit future calculations, even create inconsistency

In [24]:
#converting unit_price to float for consistency in incremental 
inc_df['unit_price'] = inc_df['unit_price'].astype(float)
print(inc_df['unit_price'].dtype)#to confirm the conversion to float

float64


## T4:CREATING TOTAL PRICE COLUMN
- quantity * unit price = total price


In [25]:
#Calculating total price for full dataset
raw_df['total_price'] = raw_df['quantity'] * raw_df['unit_price']#calculating total price by multiplying quantity and unit price
#Calculating total price for incremental dataset
inc_df['total_price'] = inc_df['quantity'] * inc_df['unit_price']#calculating total price by multiplying quantity and unit price

display(raw_df[['quantity', 'unit_price', 'total_price']].head())#displaying the first few rows of the raw dataset with quantity, unit price and total price
display(inc_df[['quantity', 'unit_price', 'total_price']].head())#displaying the first few rows of the incremental dataset with quantity, unit price and total price


,quantity,unit_price,total_price
0,2.0,500.0,1000.0
1,2.0,500.0,1000.0
2,2.0,250.0,500.0
3,2.0,750.0,1500.0
4,3.0,500.0,1500.0


,quantity,unit_price,total_price
0,1.5,900.0,1350.0
1,1.0,300.0,300.0
2,1.0,600.0,600.0
3,1.5,300.0,450.0
4,2.0,600.0,1200.0


## T5: DROPPING ROWS WITH MISSING VALUES(ORDER_ID AND PRODUCT)

In [26]:
#before dropping rows
print("rows before drop full:", len(raw_df))
print("rows before drop inc:", len(inc_df))
#dropping rows where order_id or product is missing
raw_df = raw_df.dropna(subset=['order_id', 'product'])
inc_df = inc_df.dropna(subset=['order_id', 'product'])

#after dropping rows
print("Rows after drop  full:", len(raw_df))
print("Rows after drop inc:", len(inc_df))


rows before drop full: 100
rows before drop inc: 10
Rows after drop  full: 100
Rows after drop inc: 10


since we didn't have any rows to drop before and after we move on to the next transformation

## T5: REMOVING DUPLICATES
full dataset has one duplicate row

In [27]:
print("Full dataset shape before:", raw_df.shape)#show number of rows before removing duplicates

raw_df = raw_df.drop_duplicates()# removing duplicates from the raw dataset
print("Full dataset shape after:", raw_df.shape)# Show number of rows after


Full dataset shape before: (100, 8)
Full dataset shape after: (99, 8)


no duplicates after transformation since we have 99 rows

## T6:CATEGORIZING REGION AS KNOWN AND UNKNOWN

In [28]:
#fill missing regions that i gave as a placeholder with "Unknown" that i gave as a placeholder
raw_df['region'] = raw_df['region'].fillna("Unknown")#full dataset
inc_df['region'] = inc_df['region'].fillna("Unknown")#incremental dataset

#creating region_status column for Known vs Unknown values
raw_df['region_status'] = raw_df['region'].apply(lambda x: 'Unknown' if x == 'Unknown' else 'Known')#full dataset
inc_df['region_status'] = inc_df['region'].apply(lambda x: 'Unknown' if x == 'Unknown' else 'Known')#incremental dataset
# Displaying the first few rows of the region and region_status columns in both datasets
print(raw_df[['region', 'region_status']].head())
print(inc_df[['region', 'region_status']].head())


    region region_status
0    South         Known
1    North         Known
2  Unknown       Unknown
3     West         Known
4    South         Known
    region region_status
0  Central         Known
1  Central         Known
2  Central         Known
3  Central         Known
4    North         Known


region status just shows the regions that are known and the ones that are not the missing values

## Saving the transformed files

In [29]:
raw_df.to_csv("transformed/transformed_full.csv", index=False)
inc_df.to_csv("transformed/transformed_incremental.csv", index=False)
